In [ ]:
test version clinet 3 cuurently running 



In [1]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import os
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("DP_SA_FraminghamClient")

CLIENT_CONFIG = {
    "local_epochs": 3,
    "batch_size": 32,
    "learning_rate": 0.001,
    "weight_decay": 1e-5,
    "dropout_rate": 0.3,
    "server_address": "localhost:8080",
    "proximal_mu": 0.01,

    # Differential Privacy parameters
    "dp_enabled": True,
    "dp_noise_multiplier": 1.0,
    "dp_max_grad_norm": 1.0,
    "dp_epsilon": 8.0,
    "dp_delta": 1e-5,

    # Secure Aggregation parameters
    "sa_enabled": True,
    "sa_noise_scale": 1e-3,

    # Additional security flags
    "smpc_enabled": False,
    "he_enabled": False,
}

class DifferentialPrivacyMechanism:
    def __init__(self, noise_multiplier, max_grad_norm, epsilon, delta):
        self.noise_multiplier = noise_multiplier
        self.max_grad_norm = max_grad_norm
        self.epsilon = epsilon
        self.delta = delta
        logger.info(f"DP initialized: ε={epsilon}, δ={delta}, noise_multiplier={noise_multiplier}")

    def clip_gradients(self, model):
        total_norm = 0.0
        for param in model.parameters():
            if param.grad is not None:
                param_norm = param.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5

        clip_coef = self.max_grad_norm / (total_norm + 1e-6)
        if clip_coef < 1:
            for param in model.parameters():
                if param.grad is not None:
                    param.grad.data.mul_(clip_coef)
        return total_norm

    def add_noise(self, model):
        noise_scale = self.noise_multiplier * self.max_grad_norm
        for param in model.parameters():
            if param.grad is not None:
                noise = torch.normal(0, noise_scale, size=param.grad.shape, device=param.grad.device)
                param.grad.data.add_(noise)

class SecureAggregation:
    def __init__(self, noise_scale=1e-3):
        self.noise_scale = noise_scale

    def add_random_mask(self, parameters):
        masked_params = []
        for param in parameters:
            mask = np.random.normal(0, self.noise_scale, param.shape)
            masked_params.append(param + mask)
        return masked_params

class PrivacyEnhancedHeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(CLIENT_CONFIG["dropout_rate"]),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

class PrivacyAwareFedProxLoss(nn.Module):
    def __init__(self, base_criterion, mu=0.01):
        super().__init__()
        self.base_criterion = base_criterion
        self.mu = mu

    def forward(self, y_pred, y_true, model_params, global_params):
        base_loss = self.base_criterion(y_pred, y_true)
        proximal_term = 0.0
        if global_params is not None:
            for local_param, global_param in zip(model_params, global_params):
                proximal_term += torch.sum((local_param - global_param) ** 2)
            loss = base_loss + (self.mu / 2) * proximal_term
        else:
            loss = base_loss
        return loss

def load_data(data_path):
    try:
        df = pd.read_csv(data_path)
        logger.info(f"Loaded {data_path} with shape {df.shape}")
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            logger.info(f"Dropping {missing_values} rows with missing data")
            df.dropna(inplace=True)
            logger.info(f"Shape after dropping missing data: {df.shape}")
        if "TenYearCHD" not in df.columns:
            raise ValueError("Target column 'TenYearCHD' missing")
        X = df.drop(columns=["TenYearCHD"])
        y = df["TenYearCHD"]
        logger.info(f"Class distribution: {y.value_counts().to_dict()}")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=CLIENT_CONFIG["batch_size"], shuffle=True)
        return dataloader, X.shape[1]
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

class DP_SA_Client(fl.client.NumPyClient):
    def __init__(self, model, dataloader, device, client_id):
        self.model = model
        self.dataloader = dataloader
        self.device = device
        self.client_id = client_id
        self.global_params = None

        self.dp_enabled = CLIENT_CONFIG["dp_enabled"]
        if self.dp_enabled:
            self.dp_mechanism = DifferentialPrivacyMechanism(
                CLIENT_CONFIG["dp_noise_multiplier"],
                CLIENT_CONFIG["dp_max_grad_norm"],
                CLIENT_CONFIG["dp_epsilon"],
                CLIENT_CONFIG["dp_delta"]
            )

        self.sa_enabled = CLIENT_CONFIG["sa_enabled"]
        if self.sa_enabled:
            self.secure_agg = SecureAggregation(noise_scale=CLIENT_CONFIG["sa_noise_scale"])

    def get_parameters(self, config):
        params = [val.detach().cpu().numpy() for val in self.model.parameters()]

        if self.sa_enabled:
            params = self.secure_agg.add_random_mask(params)
            logger.info(f"Client {self.client_id} applied Secure Aggregation masking")

        return params

    def set_parameters(self, parameters):
        self.global_params = [torch.tensor(p, device=self.device) for p in parameters]
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v, device=self.device) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)
        logger.info(f"Client {self.client_id}: parameters updated from server")

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        criterion = nn.BCELoss()
        proximal_criterion = PrivacyAwareFedProxLoss(criterion, mu=CLIENT_CONFIG["proximal_mu"])
        optimizer = torch.optim.Adam(self.model.parameters(), lr=CLIENT_CONFIG["learning_rate"], weight_decay=CLIENT_CONFIG["weight_decay"])

        total_loss, total_samples, correct = 0.0, 0, 0

        for epoch in range(CLIENT_CONFIG["local_epochs"]):
            epoch_loss, epoch_samples = 0.0, 0
            for batch_idx, (X, y) in enumerate(self.dataloader):
                X, y = X.to(self.device), y.to(self.device)
                y_pred = self.model(X)
                loss = proximal_criterion(y_pred, y, self.model.parameters(), self.global_params)
                optimizer.zero_grad()
                loss.backward()

                if self.dp_enabled:
                    grad_norm = self.dp_mechanism.clip_gradients(self.model)
                    self.dp_mechanism.add_noise(self.model)
                    if batch_idx % 10 == 0:
                        logger.info(f"Client {self.client_id} DP applied: grad_norm={grad_norm:.4f}")

                optimizer.step()

                batch_loss = loss.item() * X.size(0)
                total_loss += batch_loss
                epoch_loss += batch_loss
                total_samples += X.size(0)
                epoch_samples += X.size(0)

                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()

                if batch_idx % 5 == 0:
                    logger.info(f"Client {self.client_id} - Epoch {epoch+1}/{CLIENT_CONFIG['local_epochs']} - Batch {batch_idx}/{len(self.dataloader)} - Loss: {loss.item():.4f}")
            logger.info(f"Client {self.client_id} - Epoch {epoch+1} complete - Loss: {epoch_loss/epoch_samples:.4f}")

        avg_loss = total_loss / total_samples if total_samples else 0
        accuracy = correct / total_samples if total_samples else 0

        logger.info(f"Client {self.client_id} training finished - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

        # Send security flags back in metrics including client ID
        return self.get_parameters({}), total_samples, {
            "loss": avg_loss,
            "accuracy": accuracy,
            "dp_enabled": int(self.dp_enabled),
            "sa_enabled": int(self.sa_enabled),
            "smpc_enabled": int(CLIENT_CONFIG.get("smpc_enabled", False)),
            "he_enabled": int(CLIENT_CONFIG.get("he_enabled", False)),
            "client_id": self.client_id  # Send client ID here
        }

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        criterion = nn.BCELoss()

        loss, total, correct = 0.0, 0, 0

        with torch.no_grad():
            for X, y in self.dataloader:
                X, y = X.to(self.device), y.to(self.device)
                y_pred = self.model(X)
                batch_loss = criterion(y_pred, y).item()
                loss += batch_loss * X.size(0)
                total += X.size(0)
                predicted = (y_pred > 0.5).float()
                correct += (predicted == y).sum().item()

        avg_loss = loss / total if total else 0
        accuracy = correct / total if total else 0

        logger.info(f"Client {self.client_id} evaluation - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

        # Send security flags back in metrics including client ID
        return float(avg_loss), total, {
            "accuracy": accuracy,
            "dp_enabled": int(self.dp_enabled),
            "sa_enabled": int(self.sa_enabled),
            "smpc_enabled": int(CLIENT_CONFIG.get("smpc_enabled", False)),
            "he_enabled": int(CLIENT_CONFIG.get("he_enabled", False)),
            "client_id": self.client_id  # Send client ID here
        }

def start_client(client_id=0, server_address=None):
    if server_address:
        CLIENT_CONFIG["server_address"] = server_address

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    data_path = f"framingham_part{client_id+1}.csv"
    if not os.path.exists(data_path):
        logger.error(f"Data file {data_path} not found")
        return

    dataloader, input_size = load_data(data_path)

    model = PrivacyEnhancedHeartDiseaseModel(input_size=input_size).to(device)
    logger.info(f"Model initialized with input size {input_size}")

    client = DP_SA_Client(model, dataloader, device, client_id)

    print(f"\n===== DP+SA Framingham FL Client {client_id} =====")
    print(f"Server:              {CLIENT_CONFIG['server_address']}")
    print(f"Data file:           {data_path}")
    print(f"Local epochs:        {CLIENT_CONFIG['local_epochs']}")
    print(f"Batch size:          {CLIENT_CONFIG['batch_size']}")
    print(f"Device:              {device}")
    print("==============================================\n")

    fl.client.start_client(server_address=CLIENT_CONFIG["server_address"], client=client)

if __name__ == "__main__":
    import sys
    import argparse

    if 'ipykernel' in sys.modules:
        start_client(client_id=2)
    else:
        parser = argparse.ArgumentParser(description="DP+SA Framingham FL Client")
        parser.add_argument("--id", type=int, default=0, help="Client ID")
        parser.add_argument("--server", type=str, default="localhost:8080", help="Server address")
        parser.add_argument("--disable-sa", action="store_true", help="Disable Secure Aggregation masking")
        parser.add_argument("--dp-epsilon", type=float, default=8.0, help="DP epsilon parameter")
        parser.add_argument("--dp-noise", type=float, default=1.0, help="DP noise multiplier")
        args = parser.parse_args()

        CLIENT_CONFIG["dp_epsilon"] = args.dp_epsilon
        CLIENT_CONFIG["dp_noise_multiplier"] = args.dp_noise
        CLIENT_CONFIG["sa_enabled"] = not args.disable_sa

        start_client(args.id, args.server)


2025-05-24 22:10:01,328 - DP_SA_FraminghamClient - INFO - Using device: cuda
2025-05-24 22:10:01,366 - DP_SA_FraminghamClient - INFO - Loaded framingham_part3.csv with shape (1060, 16)
2025-05-24 22:10:01,372 - DP_SA_FraminghamClient - INFO - Class distribution: {0: 898, 1: 162}
2025-05-24 22:10:01,503 - DP_SA_FraminghamClient - INFO - Model initialized with input size 15
2025-05-24 22:10:01,504 - DP_SA_FraminghamClient - INFO - DP initialized: ε=8.0, δ=1e-05, noise_multiplier=1.0
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
2025-05-24 22:10:01,505 - flwr - WARNING - DEPRECATED FEATURE: flwr.client.start_client() is deprecated.
	Instead, use the `flower-supernode


===== DP+SA Framingham FL Client 2 =====
Server:              localhost:8080
Data file:           framingham_part3.csv
Local epochs:        3
Batch size:          32
Device:              cuda



INFO :      
2025-05-24 22:10:02,032 - flwr - INFO - 
INFO :      Received: evaluate message 02e5129a-20e1-45c4-b122-cbb1e56fb490
2025-05-24 22:10:02,034 - flwr - INFO - Received: evaluate message 02e5129a-20e1-45c4-b122-cbb1e56fb490
2025-05-24 22:10:02,037 - flwr - WARNING - Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
2025-05-24 22:10:02,085 - DP_SA_FraminghamClient - INFO - Client 2: parameters updated from server
2025-05-24 22:10:02,221 - DP_SA_FraminghamClient - INFO - Client 2 evaluation - Loss: 0.7014, Accuracy: 0.8472
INFO :      Sent reply
2025-05-24 22:10:02,224 - flwr - INFO - Sent reply
INFO :      
2025-05-24 22:10:02,243 - flwr - INFO - 
INFO :      Received: train message dcc7455b-2715-4bd9-8808-f7874678b15d
2025-05-24 22:10:02,247 - flwr - INFO - Received: train message dcc7455b-2715-4bd9-8808-f7874678b15d
2025-05-24